In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import gkey

# urls
places_api = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"

# data paths
city_data_path = "../output_data/cities.csv"

In [2]:
city_data = pd.read_csv(city_data_path)

city_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Sarangani,25,PH,1579993378,79,5.40,125.46,82.00,4.18
1,Gotsucho,100,JP,1579993379,70,35.00,132.22,50.09,6.97
2,Bethel,90,US,1579993379,100,41.37,-73.41,51.01,2.60
3,Jamestown,100,US,1579993380,93,42.10,-79.24,36.00,0.89
4,Ushuaia,100,AR,1579993380,82,-54.80,-68.30,40.06,2.43
...,...,...,...,...,...,...,...,...,...
554,Tonota,0,BW,1579993577,56,-21.48,27.48,71.47,0.98
555,Owando,86,CG,1579993577,38,-0.48,15.90,71.51,0.86
556,Roald,100,NO,1579993578,90,62.58,6.13,48.00,5.36
557,Twentynine Palms,1,US,1579993578,23,34.14,-116.05,78.80,3.60


In [3]:
# gmaps configuration
gmaps.configure(api_key=gkey)

In [4]:
# function for heatmap1
def heatmap1(locations, weights):
    fig = gmaps.figure()
    heat_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False,)
    fig.add_layer(heat_layer) 
    fig   
    return fig

# function for heatmap2
def heatmap2(humidity_locations, hotel_locations, humidity, hotel_info, hover_text):
    fig = gmaps.figure()
    
    heat_layer = gmaps.heatmap_layer(humidity_locations, weights=humidity, dissipating=False,)
    marker_layer = gmaps.marker_layer(hotel_locations, hover_text=hover_text)
    
    fig.add_layer(heat_layer) 
    fig.add_layer(marker_layer)
    
    fig  
    
    return fig

In [5]:
locations = city_data[["Lat", "Lng"]]
weights = city_data["Humidity"]
heatmap1(locations, weights)

Figure(layout=FigureLayout(height='420px'))

In [6]:
vacation_cities = city_data.dropna()

cold = vacation_cities.loc[vacation_cities["Max Temp"] < 65].index
vacation_cities = vacation_cities.drop(cold)

hot = vacation_cities.loc[vacation_cities["Max Temp"] > 85].index
vacation_cities = vacation_cities.drop(hot)

windy = vacation_cities.loc[vacation_cities["Wind Speed"] > 10].index
vacation_cities = vacation_cities.drop(windy)

cloudy = vacation_cities.loc[vacation_cities["Cloudiness"] > 0].index
vacation_cities = vacation_cities.drop(cloudy)

humid = vacation_cities.loc[vacation_cities["Humidity"] > 60].index
vacation_cities = vacation_cities.drop(humid)

hotel_df = vacation_cities

hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
59,Broken Hill,0,AU,1579993399,30,-31.95,141.43,80.67,5.98
194,Guerrero Negro,0,MX,1579993422,53,27.98,-114.06,67.24,4.61
202,Lovington,0,US,1579993424,20,32.94,-103.35,66.00,5.36
286,Kontagora,0,NG,1579993381,14,10.40,5.47,69.98,3.96
325,Acajutla,0,SV,1579993492,56,13.59,-89.83,82.00,0.89
414,Rawson,0,AR,1579993523,37,-43.30,-65.10,76.14,6.69
420,Jardim,0,BR,1579993525,57,-21.48,-56.14,74.70,1.99
449,Presidencia Roque Saenz Pena,0,AR,1579993539,43,-26.79,-60.44,79.27,1.91
554,Tonota,0,BW,1579993577,56,-21.48,27.48,71.47,0.98


In [7]:
hotel_list = []
address_list = []
term = "Hotel"
inputtype = "textquery"
radius = "10"
fields = ["name", "formatted_address"]

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    locationbias = f"circle:{radius}@{lat},{lng}"
    target_url = f"{places_api}key={gkey}&input={term}&inputtype={inputtype}&locationbias=circle:{radius}@{lat},{lng}&fields=name,formatted_address"
    response = requests.get(target_url).json()
    hotel_list.append(response["candidates"][0]["name"])
    address_list.append(response["candidates"][0]["formatted_address"])

hotel_df["Hotel Name"] = hotel_list
hotel_df["Hotel Address"] = address_list

hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Address
59,Broken Hill,0,AU,1579993399,30,-31.95,141.43,80.67,5.98,Desert Sand Motor Inn,"357 Cobalt St, Broken Hill NSW 2880, Australia"
194,Guerrero Negro,0,MX,1579993422,53,27.98,-114.06,67.24,4.61,The Halfway Inn,"Paralelo 28, México 1, 23940 Guerrero Negro, B..."
202,Lovington,0,US,1579993424,20,32.94,-103.35,66.00,5.36,Zia Park Casino Hotel & Racetrack,"3901 W Millen Dr, Hobbs, NM 88240, United States"
286,Kontagora,0,NG,1579993381,14,10.40,5.47,69.98,3.96,Safara Motel,"Niger, Nigeria"
325,Acajutla,0,SV,1579993492,56,13.59,-89.83,82.00,0.89,Royal Decameron Salinitas,"Carretera al Litoral Km. 84 Acajutla, Sonsonat..."
414,Rawson,0,AR,1579993523,37,-43.30,-65.10,76.14,6.69,Hotel Punta León,"Juan de la Piedra 210, U9103 Playa Union, Chub..."
420,Jardim,0,BR,1579993525,57,-21.48,-56.14,74.70,1.99,Vitória Hotel,"R. Fábio Martins Barbosa, 1 - Centro, Jardim -..."
449,Presidencia Roque Saenz Pena,0,AR,1579993539,43,-26.79,-60.44,79.27,1.91,Hotel Orel,"Saavedra 485, H3700BJU Sáenz Peña, Chaco, Arge..."
554,Tonota,0,BW,1579993577,56,-21.48,27.48,71.47,0.98,Baraka Valley Lodge,"A1 Road, Francistown, Botswana"


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
humidity_locations = city_data[["Lat", "Lng"]]
weight = city_data["Humidity"]
hover_text = hotel_list

# Display Map
heatmap2(humidity_locations, hotel_locations, weight, hotel_info, hover_text)

Figure(layout=FigureLayout(height='420px'))